In [77]:
import os, sys, glob
import pandas as pd
import numpy as np

path = '../test_dataset/'
        
def find_components(file):
    data = pd.read_csv(file)
    data.columns.values[0] = 'voxel_id'
    data = data[['voxel_id', 'x', 'y', 'z']]
    data['visited'] = False
    component_id = 1
    path_to_save = file[:-4]
    result_data = pd.DataFrame(columns=['component_id', 'total_voxels', 'internal_voxels', 'external_voxels', '%_of_internal_voxels', '%_of_external_voxels', 'voxels'])
    f = open(path_to_save + '_non_zero_percentage_of_interior_voxels.txt', 'w+')
    
    def label_components(voxel_id):
        if data.at[voxel_id, 'visited'] == False:
            data.at[voxel_id, 'visited'] = True
            result_data.at[component_id, 'voxels'].append(voxel_id)
            voxel = data.loc[voxel_id]
            neighbor_voxels = data[(abs(voxel['x'] - data['x']) + abs(voxel['y'] - data['y']) + abs(voxel['z'] - data['z'])) == 1].index
            if len(neighbor_voxels) == 6:
                nonlocal internal_voxels
                internal_voxels += 1
            for neighbor_voxel_id in neighbor_voxels:
                label_components(neighbor_voxel_id)
    
    for voxel_id in data.index:
        if data.at[voxel_id, 'visited'] == False:
            result_data.at[component_id, 'voxels'] = []
            internal_voxels = 0
            label_components(voxel_id)
            result_data.at[component_id, 'voxels'].sort()
            total_voxels = len(result_data.at[component_id, 'voxels'])
            result_data.at[component_id, 'total_voxels'] = total_voxels
            external_voxels = total_voxels - internal_voxels
            result_data.at[component_id, 'component_id'] = component_id
            result_data.at[component_id, 'internal_voxels'] = internal_voxels
            result_data.at[component_id, 'external_voxels'] = external_voxels
            result_data.at[component_id, '%_of_internal_voxels'] = round((internal_voxels / total_voxels) * 100, 2)
            result_data.at[component_id, '%_of_external_voxels'] = round((external_voxels / total_voxels) * 100, 2)
            # create a text file with non-zero internals
            if internal_voxels > 0:
                f.write(str(result_data.loc[component_id]) + '\n\n\n')
            component_id += 1
    
    f.close()
    result_data.to_csv(path_to_save + '_connected_components.csv')    
            
for file in sorted(glob.glob(os.path.join(path, '**/*.csv'), recursive=True)):
    find_components(file)